In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [2]:
batch_size=64 #Batch size for training.
epochs=100 #No of epochs for training.
latient_dim=256 #latient dimentilaty of the encoding space.
num_samples=5000 #No of samples to train on
data_path="hin.txt" #path to the txt file on the disk.

In [3]:
# Vectorize the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,"r", encoding="utf-8") as file:
    lines = file.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # we use "tab" as the "start sequence" character
    # for the targets and "\n" as "end sequence" character.
    target_text = "\t" + target_text +"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
# input_texts
len(target_characters)

87

In [5]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [6]:
print("Number of samples:", len(input_texts))
print("Number of unique input_tokens:", num_encoder_tokens)
print("Number of unique output_tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 2909
Number of unique input_tokens: 70
Number of unique output_tokens: 87
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
# input_token_index, target_token_index

In [10]:
encoder_input_data=np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data=np.zeros((len(input_texts), max_encoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data=np.zeros((len(input_texts), max_encoder_seq_length, num_decoder_tokens), dtype="float32")

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]]=1.
    encoder_input_data[i, t+1:, input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
        # decoders_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t-1:, target_token_index[char]]=1.
        decoder_input_data[i, t+1:, target_token_index[' ']]=1.        
        decoder_target_data[i, t:, target_token_index[' ']]=1.
        

In [15]:
encoder_input_data[9].shape

(107, 70)

In [16]:
# define the input sequence and process it.
encoder_inputs=Input(shape=(None, num_encoder_tokens))
encoder=LSTM(latient_dim,return_state=True)
encoder_ouptuts, state_h, state_c = encoder(encoder_inputs)
# We discard "encoder_outputs" and only keep the states.
encoder_states = [state_h, state_c]

In [17]:
# set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we set up our decoder to return full output sequences,
# and return internal states as well. we don't use the
# return states in the training model, but we will use them in infrerence.
decoder_lstm = LSTM(latient_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)

In [21]:
# Define the model that will turn
# encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# run training
model.compile(optimizer='rmsprop',loss="categorical_crossentropy", metrics=["accuracy"])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,epochs=10, validation_split=0.2)


Epoch 1/10
37/37 [==============================] - 24s 545ms/step - loss: 258.1322 - accuracy: 0.2607 - val_loss: 374.5443 - val_accuracy: 0.4365
Epoch 2/10
37/37 [==============================] - 19s 509ms/step - loss: 260.5521 - accuracy: 0.2607 - val_loss: 378.5403 - val_accuracy: 0.4365
Epoch 3/10
37/37 [==============================] - 19s 505ms/step - loss: 263.1492 - accuracy: 0.2607 - val_loss: 381.6819 - val_accuracy: 0.4365
Epoch 4/10
37/37 [==============================] - 19s 510ms/step - loss: 265.5109 - accuracy: 0.2607 - val_loss: 386.2249 - val_accuracy: 0.4365
Epoch 5/10
37/37 [==============================] - 23s 626ms/step - loss: 268.1894 - accuracy: 0.2607 - val_loss: 389.3379 - val_accuracy: 0.4365
Epoch 6/10
37/37 [==============================] - 23s 623ms/step - loss: 270.5773 - accuracy: 0.2607 - val_loss: 392.8784 - val_accuracy: 0.4365
Epoch 7/10
37/37 [==============================] - 23s 624ms/step - loss: 272.9899 - accuracy: 0.2607 - val_loss: 396

In [24]:
# Next: inference mode (sampling)
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initials state
# and a "start of sequence" tocken as target.
# Output will be the next target taken 
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latient_dim,))
decoder_state_input_c = Input(shape=(latient_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states=[state_h, state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)

# Reverse-lookup taken index to decode sequence back to something readable
reverse_input_char_index = dict((i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char, i in target_token_index.items())

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0,0, target_token_index['\t']]=1.
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition=False
    decoded_sentence=''
    while not stop_condition:
        output_tokens, h, c=decoder_model.predict([target_seq]+states_value)
        
        # Sample a tocken
        sampled_token_index = np.argmax(output_tokens[0, -1,:])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # Exit condition: either hit max length or find stop character.
        if (sampled_char=="\n" or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition=True
            
        # Update the target sequence of length 1.
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index]=1.
        
        # Update states.
        states_value=[h,c]
    return decoded_sentence

In [36]:
for seq_index in range(100):
    # Tocken one sequence (Part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decode_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:",input_texts[seq_index])
    print("Decode sentence:",decode_sentence)
    

1/1 [==============================] - 0s 19ms/step


KeyboardInterrupt: 